In [225]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from sklearn.metrics import confusion_matrix
from tensorflow.keras import layers, regularizers

# Carga de datos

In [226]:
data = pd.read_csv('Maternal Health Risk Data Set.csv')
data['RiskLevel'] = data['RiskLevel'].map({'low risk': 0, 'mid risk': 1, 'high risk': 2})
data['PulsePressure'] = data['SystolicBP'] - data['DiastolicBP']
data['MAP'] = data['DiastolicBP'] + (data['PulsePressure'] / 3)
data['BPRatio'] = data['SystolicBP'] / data['DiastolicBP']
data['SBP/Age'] = data['SystolicBP'] / data['Age']
data['DBP/Age'] = data['DiastolicBP'] / data['Age']
data['BS/Age'] = data['BS'] / data['Age']
data.drop(columns=(['HeartRate', 'DiastolicBP']), inplace=True)
x = data.drop(columns=['RiskLevel'])
y = data['RiskLevel']
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.15, random_state=10)
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)
data

,Age,SystolicBP,BS,BodyTemp,RiskLevel,PulsePressure,MAP,BPRatio,SBP/Age,DBP/Age,BS/Age
0,25,130,15.0,98.0,2,50,96.666667,1.625000,5.200000,3.200000,0.600000
1,35,140,13.0,98.0,2,50,106.666667,1.555556,4.000000,2.571429,0.371429
2,29,90,8.0,100.0,2,20,76.666667,1.285714,3.103448,2.413793,0.275862
3,30,140,7.0,98.0,2,55,103.333333,1.647059,4.666667,2.833333,0.233333
4,35,120,6.1,98.0,0,60,80.000000,2.000000,3.428571,1.714286,0.174286
...,...,...,...,...,...,...,...,...,...,...,...
1009,22,120,15.0,98.0,2,60,80.000000,2.000000,5.454545,2.727273,0.681818
1010,55,120,18.0,98.0,2,30,100.000000,1.333333,2.181818,1.636364,0.327273
1011,35,85,19.0,98.0,2,25,68.333333,1.416667,2.428571,1.714286,0.542857
1012,43,120,18.0,98.0,2,30,100.000000,1.333333,2.790698,2.093023,0.418605


In [227]:
classes = np.unique(y_train)
class_weights_array = compute_class_weight(
    class_weight='balanced',
    classes=classes,
    y=y_train
)
class_weights = dict(zip(classes, class_weights_array))

model = tf.keras.Sequential([

    tf.keras.layers.Dense(
        64, activation='relu',
        input_shape=(x_train.shape[1],),
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(
        32, activation='relu',
        kernel_regularizer=tf.keras.regularizers.l2(0.001)
    ),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(8, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
    tf.keras.layers.BatchNormalization(),

    tf.keras.layers.Dense(3, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    x_train, y_train,
    epochs=200,
    batch_size=32,
    validation_data=(x_test, y_test),
    class_weight=class_weights,
    verbose=2
)

y_pred_train = model.predict(x_train)
y_pred_test = model.predict(x_test)

y_pred_train_classes = np.argmax(y_pred_train, axis=1)
y_pred_test_classes = np.argmax(y_pred_test, axis=1)


Epoch 1/200


/Users/sofia/Desktop/séptimo/ProyectoFinalMIA/venv/lib/python3.13/site-packages/keras/src/layers/core/dense.py:95: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


27/27 - 1s - 33ms/step - accuracy: 0.4599 - loss: 1.2121 - val_accuracy: 0.3595 - val_loss: 1.1316
Epoch 2/200
27/27 - 0s - 2ms/step - accuracy: 0.4913 - loss: 1.0134 - val_accuracy: 0.5033 - val_loss: 1.0404
Epoch 3/200
27/27 - 0s - 1ms/step - accuracy: 0.5854 - loss: 0.8809 - val_accuracy: 0.6013 - val_loss: 0.9690
Epoch 4/200
27/27 - 0s - 2ms/step - accuracy: 0.6144 - loss: 0.8244 - val_accuracy: 0.6471 - val_loss: 0.9140
Epoch 5/200
27/27 - 0s - 1ms/step - accuracy: 0.6307 - loss: 0.7895 - val_accuracy: 0.7190 - val_loss: 0.8700
Epoch 6/200
27/27 - 0s - 1ms/step - accuracy: 0.6481 - loss: 0.7904 - val_accuracy: 0.6732 - val_loss: 0.8391
Epoch 7/200
27/27 - 0s - 1ms/step - accuracy: 0.6655 - loss: 0.7593 - val_accuracy: 0.6863 - val_loss: 0.8013
Epoch 8/200
27/27 - 0s - 1ms/step - accuracy: 0.6760 - loss: 0.7328 - val_accuracy: 0.6863 - val_loss: 0.7729
Epoch 9/200
27/27 - 0s - 1ms/step - accuracy: 0.6760 - loss: 0.7437 - val_accuracy: 0.7516 - val_loss: 0.7506
Epoch 10/200
27/27 - 

In [228]:
y_new_pred_train = model.predict(x_train)
y_new_pred_test = model.predict(x_test)
y_new_pred_train_classes = np.argmax(y_new_pred_train, axis=1)
y_new_pred_test_classes = np.argmax(y_new_pred_test, axis=1)

A = confusion_matrix(y_train, y_new_pred_train_classes)
num_classes = A.shape[0]

TP = np.diag(A)
FP = np.sum(A, axis=0) - TP
FN = np.sum(A, axis=1) - TP
TN = np.sum(A) - (TP + FP + FN)

# Precision: TP / (TP + FP)
precision_per_class = TP / (TP + FP + 1e-12)

# Recall: TP / (TP + FN)
recall_per_class = TP / (TP + FN + 1e-12)

# Accuracy
accuracy = np.sum(TP) / np.sum(A)

# Macro averages
precision_macro = np.mean(precision_per_class)
recall_macro = np.mean(recall_per_class)

print("\n=== TRAIN METRICS ===")
print("Accuracy:", accuracy)

print("\nPrecision per class:", precision_per_class)
print("Macro Precision:", precision_macro)

print("\nRecall per class:", recall_per_class)
print("Macro Recall:", recall_macro)

27/27 ━━━━━━━━━━━━━━━━━━━━ 0s 421us/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step

=== TRAIN METRICS ===
Accuracy: 0.8478513356562137

Precision per class: [0.85797101 0.79925651 0.88663968]
Macro Precision: 0.8479557320607709

Recall per class: [0.84813754 0.76241135 0.95217391]
Macro Recall: 0.8542409321259394


In [229]:
A = confusion_matrix(y_test, y_new_pred_test_classes)
num_classes = A.shape[0]

TP = np.diag(A)
FP = np.sum(A, axis=0) - TP
FN = np.sum(A, axis=1) - TP
TN = np.sum(A) - (TP + FP + FN)

# Precision: TP / (TP + FP)
precision_per_class = TP / (TP + FP + 1e-12)

# Recall: TP / (TP + FN)
recall_per_class = TP / (TP + FN + 1e-12)

# Accuracy
accuracy = np.sum(TP) / np.sum(A)

# Macro averages
precision_macro = np.mean(precision_per_class)
recall_macro = np.mean(recall_per_class)

print("\n=== TEST METRICS ===")
print("Accuracy:", accuracy)

print("\nPrecision per class:", precision_per_class)
print("Macro Precision:", precision_macro)

print("\nRecall per class:", recall_per_class)
print("Macro Recall:", recall_macro)


=== TEST METRICS ===
Accuracy: 0.803921568627451

Precision per class: [0.78181818 0.73684211 0.92682927]
Macro Precision: 0.8151631851246576

Recall per class: [0.75438596 0.77777778 0.9047619 ]
Macro Recall: 0.812308549150638


In [230]:
#model.save('maternal_health_risk_model.keras')

In [231]:
def permutation_importance(model, x_valid, y_valid, metric=accuracy_score):
    baseline = metric(y_valid, np.argmax(model.predict(x_valid), axis=1))
    importances = {}

    for i, col in enumerate(range(x_valid.shape[1])):
        x_temp = x_valid.copy()
        np.random.shuffle(x_temp[:, i]) 

        score = metric(y_valid, np.argmax(model.predict(x_temp), axis=1))
        importances[col] = baseline - score

    return importances
importances = permutation_importance(model, x_test, y_test)
sorted_importances = dict(sorted(importances.items(), key=lambda x: x[1], reverse=True))
sorted_importances

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


{7: 0.2614379084967321,
 9: 0.24836601307189543,
 5: 0.20915032679738566,
 1: 0.20261437908496738,
 2: 0.20261437908496738,
 4: 0.18300653594771243,
 6: 0.16993464052287588,
 8: 0.1568627450980392,
 0: 0.12418300653594772,
 3: 0.11764705882352944}